In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm

import SimpleITK as sitk
import matplotlib.pyplot as plt

import skimage.io
import skimage.transform
from pathlib import Path


In [3]:
# root = Path("/vol/bitbucket/bh1511/data/LUNA16")
root = Path("/data/xz1919/LUNA16")
output_dir = Path("./luna16_cropped")
output_dir.mkdir(exist_ok=True, parents=True)

# Crop images

## Input

`root`: A directory of LUNA16 dataset.

Labels are read from `candidates_V2.csv`, which contains information about each nodule and non-nodule.

Non-nodules are significantly more than nodules, but we want to keep the dataset relatively balanced. To speed up, we limit the number of non-nodules generated. **For each CT scan, we limit the number of non-nodule images cropped to be no more than 2x of the number of nodule images.**

## Output

`output_dir`: A directory with

* Images cropped from the LUNA16 dataset, in which the nodule or non-nodule centers.
* Images cropped from diffusion model generated samples. Filenames are prefixed with `gen_`.
* `labels.csv`: a CSV that contains information about images cropped from the LUNA16 dataset.
    * Used by 
        * The local-global model.
            * The preprocessing script of the model drops some non-nodule samples to make the dataset balanced.
        * `mmcls.ipynb`, which generates annotation files for use by MMClassification.
* `labels_all.csv`: a CSV that contains information about all images cropped (from LUNA16 + diffusion-generated).

# Crop images from the original LUNA16 dataset



In [3]:
candidates_df = pd.read_csv(root / "candidates_V2.csv")
candidates_df.head()

,seriesuid,coordX,coordY,coordZ,class
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,68.420000,-74.480000,-288.700000,0
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-95.209361,-91.809406,-377.426350,0
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-24.766755,-120.379294,-273.361539,0
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-63.080000,-65.740000,-344.240000,0
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,52.946688,-92.688873,-241.067872,0


In [15]:
def load_image(file):
    image_itk = sitk.ReadImage(file)
    image_itk = sitk.IntensityWindowing(image_itk, 
                                        windowMinimum=-1000, windowMaximum=400, 
                                        outputMinimum=0.0, outputMaximum=255.0)
    image_itk = sitk.Cast(image_itk, sitk.sitkUInt8)
    image_arr = sitk.GetArrayFromImage(image_itk)
    origin = np.array(list(image_itk.GetOrigin()))
    space = np.array(list(image_itk.GetSpacing()))
    return image_arr, origin, space

In [16]:
image_missing_candidate_indices = []

output_candidates = []

for series_uid, series_candidates in tqdm(candidates_df.groupby("seriesuid")):
    try:
        image, origin, space = load_image(root / f"images/{series_uid}.mhd")
    except:
        print(f"Image for {series_uid} does not exist, skipping")
        image_missing_candidate_indices += list(series_candidates.index)
        continue

    # reduce the number of non-nodules
    class1_nodules = series_candidates[series_candidates["class"] == 1]
    nr_class1_nodules = len(class1_nodules)
    non_nodules = series_candidates[series_candidates["class"] == 0]
    series_candidates = pd.concat(
        [class1_nodules, non_nodules.sample(nr_class1_nodules * 2, random_state=42)])

    for i, candidate in series_candidates.iterrows():
        node_x = candidate["coordX"]     # X coordinate of the nodule
        node_y = candidate["coordY"]     # Y coordinate of the nodule
        node_z = candidate["coordZ"]     # Z coordinate of the nodule

        # nodule center (x,y,z ordering)
        center = np.array([node_x, node_y, node_z])
        # nodule center in voxel space (x,y,z ordering)
        v_center = np.rint((center - origin) / space).astype('int')

        v_x, v_y, v_z = v_center

        roi_dim = 32
        v_x_min = v_x - roi_dim // 2
        v_x_max = v_x + roi_dim // 2
        v_y_min = v_y - roi_dim // 2
        v_y_max = v_y + roi_dim // 2

        if v_x_min < 0 or v_y_min < 0:
            print("Skipping out-of-boundary")
            continue
        roi_image = image[v_z, v_y_min:v_y_max, v_x_min:v_x_max]
        roi_filename = f"{series_uid}_{v_z}_{v_x}_{v_y}.png"

        skimage.io.imsave(output_dir / roi_filename, roi_image)
        output_candidates.append((roi_filename, candidate["class"], v_x, v_y, v_z))
    


  0%|          | 0/888 [00:00<?, ?it/s]


In [ ]:
output_df = pd.DataFrame(output_candidates, columns=["filename", "is_nodule", "vX", "vY", "vZ"])
output_df["is_generated"] = 0

output_df.to_csv(output_dir / "labels.csv", index=False)
output_df.head()

,filename,class,vX,vY,vZ,is_generated
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,1,406,155,117,0
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,1,45,212,78,0
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,0,225,348,82,0
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,0,213,288,173,0
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,0,131,397,86,0


# Crop the diffusion model generated images

In [8]:
output_df = pd.read_csv(output_dir / "labels.csv")
dg_root = Path(
    "/vol/bitbucket/xz1919/diffusion-generated-images/with-nodule/RESULTS/ADE20K-SDM-256CH/images")
# dg_root = Path("/vol/bitbucket/xz1919/diffusion-generated-images/without-nodule/ADE20K-SDM-256CH/images")
gan_root = Path("/vol/bitbucket/xz1919/GAN-generated-images")

def crop_from_generated(root: Path, is_generated: int, output_df: pd.DataFrame, get_filename, filename_prefix: str = "gen", scale: int = 1):
    total_nr_missing = 0
    generated_images = []

    for i, row in tqdm(output_df[output_df["is_nodule"] == 1].iterrows()):
        series_uid = row.filename.split("_")[0]
        v_x, v_y, v_z = row.vX, row.vY, row.vZ
        generated_image_filename = root / get_filename(series_uid, v_z)

        if not generated_image_filename.exists():
            # print(f"{generated_image_filename} does not exist!")
            total_nr_missing += 1
            continue

        img = skimage.io.imread(generated_image_filename, as_gray=True)

        roi_dim = 32
        v_x_min = v_x - roi_dim // 2
        v_x_max = v_x + roi_dim // 2
        v_y_min = v_y - roi_dim // 2
        v_y_max = v_y + roi_dim // 2

        if v_x_min < 0 or v_y_min < 0:
            print("Skipping out-of-boundary")
            continue

        roi_img = img[v_y_min // scale:v_y_max // scale, v_x_min // scale:v_x_max // scale]

        if scale > 1:
            roi_img = skimage.transform.rescale(roi_img, scale)

        roi_filename = f"{filename_prefix}_{series_uid}_{v_z}_{v_x}_{v_y}.png"
        skimage.io.imsave(output_dir / roi_filename, (roi_img * 255.0).astype(np.uint8))
        generated_images.append((roi_filename, row["is_nodule"], is_generated, v_x, v_y, v_z))
    print("Number of missing images:", total_nr_missing)
    return generated_images

In [9]:
diffusion_generated_images = crop_from_generated(dg_root, 1, output_df, lambda series_uid, v_z: f"{series_uid}_{v_z}.png.png", "diffusion", 2)
gan_generated_images = crop_from_generated(gan_root, 2, output_df, lambda series_uid, v_z: f"{series_uid}_{v_z}-fake.png", "gan", 1)

1524it [00:08, 187.12it/s]


Number of missing images: 777


1524it [00:20, 73.21it/s] 

Number of missing images: 738


In [10]:
columns = ["filename", "is_nodule", "is_generated", "vX", "vY", "vZ"]

diffusion_generated_df = pd.DataFrame(diffusion_generated_images, columns=columns)
gan_generated_df = pd.DataFrame(gan_generated_images, columns=columns)

all_df = pd.concat([output_df, diffusion_generated_df, gan_generated_df]).reset_index(drop=True)

del all_df["Unnamed: 0"]
all_df.to_csv(output_dir / "labels_all.csv", index=False)